# Mobile Price Range Predictor

In [ ]:
# Checking for saved model
import joblib
saved_model = None
try:
    saved_model = joblib.load('smartphone_model.joblib')
except FileNotFoundError as e:
    print("Saved model not found, building new model...")
    print(f"Original Exception - {e}")

In [ ]:
import pandas as pd
# Loading the dataset
smartphone = pd.read_csv("smartphone_price_range_data.csv")

# Info and histogram to analyze data
smartphone.info()
smartphone.hist(bins=50, figsize=(20, 15));

In [ ]:
# Spilting the dataset
from sklearn.model_selection import train_test_split
x = smartphone.drop('price_range', axis=1)
y = smartphone['price_range']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Scaling the data in one scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

In [ ]:
# If there's no saved model create one
if saved_model == None:
    from sklearn.metrics import accuracy_score
    # Logistic Regression
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression()
    lr.fit(x_train_std, y_train)
    lr_pred = lr.predict(x_test_std)
    lr_acc = accuracy_score(y_test, lr_pred)
    lr_data = {
        'model': lr,
        'accuracy': lr_acc,
    }

    # KNN Classification
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(x_train_std, y_train)
    knn_pred = knn.predict(x_test_std)
    knn_acc = accuracy_score(y_test, knn_pred)
    knn_data = {
        'model': knn,
        'accuracy': knn_acc
    }
    
    # Decision Tree Classification
    from sklearn.tree import DecisionTreeClassifier
    dtree = DecisionTreeClassifier()
    dtree.fit(x_train_std, y_train)
    dtree_pred = dtree.predict(x_test_std)
    dtree_acc = accuracy_score(y_test, dtree_pred)
    dtree_data = {
        'model': dtree,
        'accuracy': dtree_acc
    }
    
    # Random Forest Classification
    from sklearn.ensemble import RandomForestClassifier
    rfc = RandomForestClassifier(n_estimators=200)
    rfc.fit(x_train_std, y_train)
    rfc_pred = rfc.predict(x_test_std)
    rfc_acc = accuracy_score(y_test, rfc_pred)
    rfc_data = {
        'model': rfc,
        'accuracy': rfc_acc
    }
    
    # Model with Best Accuracy
    max_acc = max(
        lr_data['accuracy'],
        knn_data['accuracy'],
        dtree_data['accuracy'],
        rfc_data['accuracy'],
    )
    max_acc_model_data = {}
    if (max_acc == lr_data['accuracy']):
        max_acc_model_data = lr_data
    elif (max_acc == knn_data['accuracy']):
        max_acc_model_data = knn_data
    elif (max_acc == dtree_data['accuracy']):
        max_acc_model_data = dtree_data
    elif (max_acc == rfc_data['accuracy']):
        max_acc_model_data = rfc_data

    saved_model = joblib.dump(max_acc_model_data['model'], 'smartphone_model.joblib')

In [ ]:
# If there's saved model, use it!
if saved_model != None:
    battery_power = int(input("Battery Power (mAh): "))
    blue = int(input("Bluetooth Yes/No (0/1): "))
    clock_speed = float(input("Processor Clock Speed: "))
    dual_sim = int(input("Dual Sim Yes/No (0/1): "))
    fc = int(input("Front Camera Megapixels: "))
    four_g = int(input("4G Nework Yes/No (0/1): "))
    int_memory = int(input("Interal Memory (GB): "))
    m_dep = float(input("Mobile Depth (cm): "))
    mobile_wt = int(input("Mobile Weight (gm): "))
    n_cores = int(input("Number of cores of processor: "))
    pc = int(input("Primary Camera Megapixels: "))
    px_height = int(input("Pixel Resolution Height:"))
    px_width = int(input("Pixel Resolution Width:"))
    ram = int(input("RAM/Random Access Memory (MB): "))
    sc_h = float(input("Mobile Screen Height (cm): "))
    sc_w = float(input("Mobile Screen Width (cm): "))
    three_g = int(input("3G Network Yes/No (0/1): "))
    touch_screen = int(input("Touch Screen Yes/No (0/1): "))
    wifi = int(input("Wi-Fi Yes/No (0/1): "))
    
    data = {
        "battery_power": [battery_power],
        "blue": [blue],
        "clock_speed": [clock_speed],
        "dual_sim": [dual_sim],
        "fc": [fc],
        "four_g": [four_g],
        "int_memory": [int_memory],
        "m_dep": [m_dep],
        "mobile_wt": [mobile_wt],
        "n_cores": [n_cores],
        "pc": [pc],
        "px_height": [px_height],
        "px_width": [px_width],
        "ram": [ram],
        "sc_h": [sc_h],
        "sc_w": [sc_w],
        "three_g": [three_g],
        "touch_screen": [touch_screen],
        "wifi": [wifi],
    }
    sample = pd.DataFrame(data)
    sample = scaler.transform(sample)
    prediction = saved_model.predict(sample)[0]
    
    if prediction == 0:
        print("This smartphone falls under the low cost/budget category")
    elif prediction == 1:
        print("This smartphone falls under the medium cost/cost-effective category")
    elif prediction == 2:
        print("This smartphone falls under the high cost/quite expensive category")
    elif prediction == 3:
        print("This smartphone falls under the very high cost/very expensive category")
    else:
        print("Something went wrong...")